<div align="center">

# Physics-Informed Neural Networks (PINNs)

## Projet SOIA 6.1
</div>

Oscar Agnus, SOIA A3


<div align="center">

# 1D Heat Equation, stationary

</div>

Here, we try to simulate the 1D heat equation using a physics-informed neural network (PINN). The heat equation is given by:
$$
\nu \frac{\partial^2 u}{\partial x^2} = f(x)
$$

where $u$ is the temperature,
$x$ is the space coordinate,
$t$ is the time coordinate,
and $\nu$ is the thermal diffusivity.


In [321]:
# Change to inline or notebook if the plots aren't displayed correctly
# %matplotlib inline
%matplotlib notebook

# Imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import time
from datetime import timedelta

import os

# Weights and Biases - this is for logging the results
import wandb
# from colors import color
# from scipy.special import linestyle


def display_time(seconds):
    """Convert seconds to a time string 00h 00min 00s, for better readability."""
    delta = timedelta(seconds=seconds)
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}h {minutes:02}min {seconds:02}s"

## Problem definition
We define the problem in physical terms, and format it so that it can be used by the neural network.

For now, the variables in the following block will be used as global variables. We may try to estimate them later.

In [322]:
# Space domain
x_min = 0
x_max = 1
N_x = 1000 # Number of points in the space domain
X = np.linspace(x_min, x_max, N_x) # Discrete space domain
X_bc = np.array([x_min, x_max]) # Boundary conditions points


# Define useful parameters
nu = 1 # Thermal diffusivity
u_min = 0 # Minimum temperature
u_max = 1 # Maximum temperature

def f(x):
    """Source of the heat equation."""
    return torch.zeros_like(x)

def u_initial_condition(x):
    """Initial conditions for the heat equation."""
    return torch.zeros_like(x)

def u_boundary_condition(number_boundary, x_0, x_1, u_0=0, u_1=1):
    """Boundary conditions for the heat equation.
    The boundary conditions are repeated number_boundary times, so that not only 2 points are used for training."""
    x = torch.cat([torch.ones(number_boundary // 2) * x_0, torch.ones(number_boundary // 2) * x_1], dim=0)
    u = torch.cat([torch.ones(number_boundary // 2) * u_0, torch.ones(number_boundary // 2) * u_1], dim=0)
    return x, u

def u_exact(x_mini, x_maxi, u_mini, u_maxi, x):
    """Exact solution of the heat equation (or the finite element solution).
    The solution is given for each instant, and the space domain can be multidimensional."""
    A = (u_maxi - u_mini) / (x_maxi - x_mini) # Slope
    B = u_mini - A*x_mini # Ordinate at the origin
    u = A*x + B
    return u

## Neural network design
We define the neural network that will approximate the solution of the heat equation.

For now, we use only fully connected layers with a Tanh activation function.
We will investigate different architectures later.

In [323]:
# Define the model

n_layers  = 8 # Number of hidden layers
n_neurons = 20 # Number of neurons in the hidden layers

layers = [torch.nn.Linear(1, n_neurons)]
for i in range(n_layers):
    layers.append(torch.nn.Linear(n_neurons, n_neurons))
    layers.append(torch.nn.Tanh())
layers.append(torch.nn.Linear(n_neurons, 1))

my_model = torch.nn.Sequential(*layers)


# Checkpoint
my_model_checkpoint = my_model

# Summary of the model
# print(my_model)

## Loss functions
Here are defined the loss functions. There are 3 different loss functions:
- $L_{data}$ : loss function for the data points
- $L_{bc}$ &nbsp; &nbsp;: loss function for the boundary conditions
- $L_{pde}$ : loss function for the PDE

All of them use the Mean Squared Error (MSE) with the given data points.
Each one of them will be used with their relative weight, which represent their importance in the composite loss function $L_{composite}$.

In [324]:
# Gradient function
def grad(outputs, inputs):
    """Compute the gradient of 'outputs' with respect to 'inputs'."""
    return torch.autograd.grad(outputs = outputs, inputs = inputs, grad_outputs = torch.ones_like(outputs), create_graph = True)[0]

# Loss functions
def loss_data(u_pred, u_gt):
    """Loss function for the data points."""
    return torch.nn.MSELoss()(u_pred, u_gt)
    
def loss_bc(u_pred, u_bc):
    """Loss function for the boundary conditions."""
    return torch.nn.MSELoss()(u_pred, u_bc)

def loss_pde(u_pred, x_pde):
    """Loss function for the PDE.
    Needs to be changed in case of a different PDE."""
    u_x  = grad(u_pred, x_pde) # First order derivative
    u_xx = grad(u_x, x_pde)    # Second order derivative
    return torch.nn.MSELoss()(nu*u_xx, f(x_pde))

def loss_composite(u_gt, u_pred, u_bc, u_bc_pred, x_pde):
    """Composite loss function, combining the three loss functions."""
    # Weights for the composite loss function
    w_data = 0.0
    w_bc   = 1.0
    w_pde  = 1.0
    return w_data*loss_data(u_pred, u_gt) + w_bc*loss_bc(u_bc_pred, u_bc) + w_pde*loss_pde(u_pred, x_pde)

## Training and testing functions
Here we define the training and testing functions. We want to make a checkpoint of the model when the testing loss is minimal.

In [325]:
def train(model, optimizer, number_epoch, number_collocation, number_boundary, current_device = "cpu"):
    """Training function for the model."""

    model.train() # Set the model to training mode
    model_checkpoint = model # Checkpoint of the model

    # Collocation points
    inputs = torch.rand(number_collocation, requires_grad=True) # Collocation points, randomly distributed in the space domain
    inputs = inputs.unsqueeze(-1) # Add a dimension for the space domain
    inputs = inputs.to(current_device)    # Send the inputs to the device (GPU or CPU)

    # Boundary conditions
    inputs_bc, u_bc = u_boundary_condition(number_boundary, x_min, x_max) # Boundary conditions
    inputs_bc = inputs_bc.float().unsqueeze(-1) # Add a dimension for the space domain
    u_bc = u_bc.float().unsqueeze(-1)           # Add a dimension for the space domain
    inputs_bc = inputs_bc.to(current_device)    # Send the inputs to the device (GPU or CPU)
    u_bc = u_bc.to(current_device)              # Send the values to the device (GPU or CPU)

    # Array to store the losses during the training
    train_loss = np.array([])

    # Training loop
    for n in range(number_epoch):
        t_0 = time.time()
        optimizer.zero_grad()           # Zero the parameter gradients
        outputs = model(inputs)         # Forward pass for the collocation points
        outputs_bc = model(inputs_bc)   # Forward pass for the boundary

        loss = loss_composite(u_exact(x_min, x_max, u_min, u_max, inputs), outputs, u_bc, outputs_bc, inputs) # Compute the loss
        loss.backward()                # Backward pass
        optimizer.step()
        train_loss = np.append(train_loss, loss.item())

        # Checkpoint
        if loss.item() <= np.min(train_loss):
            model_checkpoint = model
            torch.save(model.state_dict(), "PINN_1D_heat_eq_stat_model_checkpoint.pth")

        # Display time and other stats
        deltat = []
        t_1 = time.time()
        deltat.append(t_1-t_0)
        eta_s = int(np.mean(deltat) * (number_epoch - n))
        if n % 100 == 0:
            print(f"#__Epoch [{n}/{number_epoch}] : ",
                  f"Duration: {t_1-t_0:.2f}s | ETA: {display_time(eta_s)}",
                  f"___ Current Loss : {train_loss[-1]} | Best Loss : {np.min(train_loss)}")

    return train_loss, model_checkpoint

## Model training

Check in the PC configuration if the GPU is available. Cuda needs to be installed to use the GPU with PyTorch.

This will speed up the training process.

In [326]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is cuda available: ", torch.cuda.is_available())
my_model.to(device)
print(f"Using device: {device}")

Is cuda available:  True
Using device: cuda


We then load the data and start the training process.

There isn't any testing data, because we will not use the accuracy metrics to assess the model.
The model will be evaluated by the `composite_loss` function.

The training data `u_data` should be either the exact solution, or some measured data.

In [327]:
# Training parameters
lr = 0.001
optimiser = optim.Adam(my_model.parameters(), lr=lr)
# optimiser = optim.SGD(my_model.parameters(), lr=lr)
n_epoch = 5000
n_colloc = 1000
n_bc = 1000

# Training
t0 = time.time()
training_loss, my_model_checkpoint = train(my_model, optimiser, n_epoch, n_colloc, n_bc, device)
t1 = time.time()

#__Epoch [0/5000] :  Duration: 0.04s | ETA: 00h 03min 11s ___ Current Loss : 0.539242148399353 | Best Loss : 0.539242148399353
#__Epoch [100/5000] :  Duration: 0.05s | ETA: 00h 03min 56s ___ Current Loss : 0.002474220935255289 | Best Loss : 0.0018383212154731154
#__Epoch [200/5000] :  Duration: 0.05s | ETA: 00h 03min 46s ___ Current Loss : 0.00019350920047145337 | Best Loss : 0.00019350920047145337
#__Epoch [300/5000] :  Duration: 0.03s | ETA: 00h 01min 58s ___ Current Loss : 0.0003014060202986002 | Best Loss : 0.00014711871335748583
#__Epoch [400/5000] :  Duration: 0.03s | ETA: 00h 02min 28s ___ Current Loss : 0.000553249497897923 | Best Loss : 0.00011620087752817199
#__Epoch [500/5000] :  Duration: 0.04s | ETA: 00h 03min 03s ___ Current Loss : 9.016136755235493e-05 | Best Loss : 9.016136755235493e-05
#__Epoch [600/5000] :  Duration: 0.04s | ETA: 00h 02min 36s ___ Current Loss : 7.998992077773437e-05 | Best Loss : 7.864562940085307e-05
#__Epoch [700/5000] :  Duration: 0.04s | ETA: 00h

In [328]:
# Display the results
print(f"Training loss: {np.min(training_loss)}")
print(f"Training time: {display_time(t1-t0)}")

# Log training results
# wandb.log({"training_loss": np.mean(training_loss), "training_time": t1-t0})

# Save the model
torch.save(my_model.state_dict(), "PINN_1D_heat_eq_stat_model.pth")

Training loss: 1.1429628102632705e-05
Training time: 00h 08min 09s


Here we initialise the wandb run, which will log the results of the training process.

(This is optional)

In [329]:
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="Projet SOIA 6.1 - PINN",
#
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": lr,
#     "architecture": "Custom_8_layers_20_neurons",
#     "dataset": "heat_equation_custom_solution",
#     "optimizer": "Adam",
#     "n_epoch": n_epoch,
#      "device": device,
#     "loss_function": "MSE",
#     "loss_hyperparameters": {w_data, w_bc, w_pde},
#     "space_domain_size": N_x,
#     "space_domain": [x_min, x_max],
#     }
# )

In [330]:
# Parameters for enhanced figure plots
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100  # Increasing dpi will enhance quality

In [341]:
# Plot solution

# Test points, randomly distributed in the space domain
X_test = torch.rand(N_x, requires_grad=True)*(x_max - x_min) + x_min
X_test = X_test.reshape(N_x, 1)
X_test = X_test.to(device) # Send the inputs to the device (GPU or CPU)

# Predict the solution
u_predict = my_model(X_test)
u_check = my_model_checkpoint(X_test)
u_ex = u_exact(x_min, x_max, u_min, u_max, X_test)

# Move tensors to CPU before converting to NumPy arrays
X_test = X_test.cpu().detach().numpy()
u_predict = u_predict.cpu().detach().numpy()
u_check = u_check.cpu().detach().numpy()
u_ex = u_ex.cpu().detach().numpy()

# Plot the solutions
plt.figure()
# plt.plot(X_test, u_predict, label="Predicted solution", color="orange")
plt.scatter(X_test, u_predict, label="Predicted points", marker="x", color="red")
# plt.plot(X_test, u_check, label="Predicted solution checkpoint", color="green")
plt.plot(X_test, u_ex, label="Exact solution", color="blue", linestyle="dotted")

plt.xlabel("x")
plt.ylabel("u")
plt.legend()
plt.title(f"Solution of the 1D heat equation, stationary \n Architecture : {n_layers} layers, {n_neurons} neurons/layer")
# plt.grid()
plt.show()

<IPython.core.display.Javascript object>

In [332]:
# Compute the mean squared error to assess the prediction's accuracy
mse_prediction = np.linalg.norm(u_predict - u_ex, 2)
mse_checkpoint = np.linalg.norm(u_check - u_ex, 2)
print(f"MSE for the prediction : {mse_prediction}")
print(f"MSE for the checkpoint : {mse_checkpoint}")

MSE for the prediction : 0.002930303569883108
MSE for the checkpoint : 0.002930303569883108


In [333]:
# Save the figure to directory
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
save_fig_directory = f"{parent_directory}/PINN_Plots/1D_heat_eq"
plt.savefig(f"{save_fig_directory}/PINN_1D_heat_eq_stat_linear_solution_{n_layers}_{n_neurons}.png")

In [337]:
# Plot the training loss
plt.figure()
plt.semilogy(training_loss, label=f"Optimizer : {optimiser.__class__.__name__}, lr={lr}") # Log scale
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title(f"Training loss of the 1D heat equation, stationary \n Architecture : {n_layers} layers, {n_neurons} neurons/layer \n Best loss : {np.min(training_loss)}")
plt.show()

<IPython.core.display.Javascript object>

In [335]:
# Save the figure to directory
plt.savefig(f"{save_fig_directory}/PINN_1D_heat_eq_stat_linear_loss_{n_layers}_{n_neurons}.png")